# 🎨 AI Watermark Remover - Training on Google Colab

このノートブックを使用して、Google ColabのGPUで高速にモデルを学習できます。

**重要**: ランタイム → ランタイムのタイプを変更 → GPU (T4) を選択してください

## 📦 Step 1: セットアップ

In [ ]:
# リポジトリをクローン
!git clone https://github.com/joppii/watermark.git
%cd watermark

In [ ]:
# 依存関係をインストール
!pip install -q torch torchvision numpy opencv-python Pillow scikit-image scipy tqdm tensorboard PyYAML colorama

In [ ]:
# GPU確認
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ GPU not available! Please enable GPU in Runtime settings.")

## 📁 Step 2: データのアップロード

以下の3つの方法から選択してください

### 方法1: Google Driveから（推奨）

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# データをコピー（事前にDriveの「MyDrive/watermark_images/」に画像を配置）
!mkdir -p data/original
!cp -r /content/drive/MyDrive/watermark_images/* data/original/

# 画像数を確認
import glob
images = glob.glob('data/original/**/*.png', recursive=True) + glob.glob('data/original/**/*.jpg', recursive=True)
print(f"✓ {len(images)} images loaded")

### 方法2: Zipファイルをアップロード

In [ ]:
from google.colab import files
import zipfile
import os

# Zipファイルをアップロード
print("画像のZipファイルを選択してください...")
uploaded = files.upload()

# 解凍
!mkdir -p data/original
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('data/original/')
        print(f"✓ {filename} extracted")

# 画像数を確認
import glob
images = glob.glob('data/original/**/*.png', recursive=True) + glob.glob('data/original/**/*.jpg', recursive=True)
print(f"✓ {len(images)} images loaded")

### 方法3: サンプルデータで試す（テスト用）

In [ ]:
# サンプル画像をコピー（テスト用）
!mkdir -p data/original
!cp sample/image.png data/original/test_001.png
# 同じ画像を複製してテスト
for i in range(2, 11):
    !cp sample/image.png data/original/test_{i:03d}.png

!ls -l data/original/

## 🎯 Step 3: ウォーターマークを作成

In [ ]:
# テキストウォーターマークを作成
!python src/prepare_data.py text -t "SAMPLE" -o data/watermarks/sample -n 15
!python src/prepare_data.py text -t "COPYRIGHT" -o data/watermarks/copyright -n 15

print("✓ Watermarks created")
!ls data/watermarks/*/

## 🔨 Step 4: 合成学習データを生成

In [ ]:
# 合成データセットを生成（各画像から3つのバリエーション）
!python src/prepare_data.py synthetic \
  -i data/original \
  --output-clean data/train/clean \
  --output-watermarked data/train/watermarked \
  -w data/watermarks/*/*.png \
  -n 3

# 生成されたデータを確認
import glob
clean = glob.glob('data/train/clean/*.png')
watermarked = glob.glob('data/train/watermarked/*.png')
print(f"\n✓ Training data prepared:")
print(f"  Clean images: {len(clean)}")
print(f"  Watermarked images: {len(watermarked)}")

## 🚀 Step 5: モデル学習を開始

In [ ]:
# 学習開始（GPU使用）
# バッチサイズは16-32に増やせます（GPU使用時）
!python src/train.py \
  --clean-dir data/train/clean \
  --watermarked-dir data/train/watermarked \
  --epochs 100 \
  --batch-size 16 \
  --val-split 0.1

## 📊 Step 6: TensorBoardで学習を監視（オプション）

In [ ]:
# TensorBoardを起動
%load_ext tensorboard
%tensorboard --logdir runs/watermark_removal

## 💾 Step 7: 学習済みモデルを保存

In [ ]:
# 最良モデルを確認
!ls -lh models/pretrained/*best.pth

# Google Driveに保存
!cp models/pretrained/*best.pth /content/drive/MyDrive/watermark_model_best.pth
print("✓ Model saved to Google Drive!")

In [ ]:
# またはローカルにダウンロード
from google.colab import files
import glob

best_models = glob.glob('models/pretrained/*best.pth')
if best_models:
    files.download(best_models[0])
    print(f"✓ Downloading: {best_models[0]}")

## 🧪 Step 8: 学習済みモデルをテスト

In [ ]:
# サンプル画像でテスト
import glob

best_model = glob.glob('models/pretrained/*best.pth')[0]
print(f"Using model: {best_model}")

!python src/main.py \
  -i sample/image.png \
  -o output/result_ai.png \
  --method ai \
  --model {best_model} \
  --auto-detect \
  --save-comparison

In [ ]:
# 結果を表示
from IPython.display import Image, display
import os

if os.path.exists('output/result_ai_comparison.png'):
    display(Image('output/result_ai_comparison.png'))
else:
    print("Comparison image not found. Showing result...")
    display(Image('output/result_ai.png'))

## 🔄 学習を再開する場合

In [ ]:
# チェックポイントから再開
# (Colabセッションが切れた場合など)
!python src/train.py \
  --clean-dir data/train/clean \
  --watermarked-dir data/train/watermarked \
  --resume /content/drive/MyDrive/checkpoint_epoch_50.pth \
  --epochs 100 \
  --batch-size 16

---

## 📝 メモ

- **GPU使用時のバッチサイズ**: 16-32推奨
- **メモリエラー時**: batch_sizeを8に下げる
- **学習時間**: 800枚×100エポック = 約15-25分（T4 GPU）
- **無料版の制限**: 最大12時間/セッション

詳細は [COLAB_GUIDE.md](COLAB_GUIDE.md) を参照